<a href="https://colab.research.google.com/github/pipaber/GNN/blob/main/proteinbert2GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!git clone https://github.com/nadavbra/protein_bert


Cloning into 'protein_bert'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 249 (delta 83), reused 61 (delta 61), pack-reused 152
Receiving objects: 100% (249/249), 23.44 MiB | 13.98 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [2]:
%cd protein_bert
%ls
!git submodule init
!git submodule update
!python setup.py install
!pip install biopython

/content/protein_bert
 bin/   protein_benchmarks/   proteinbert/  'ProteinBERT demo.ipynb'   README.md   setup.py
Submodule 'proteinbert/shared_utils' (https://github.com/nadavbra/shared_utils.git) registered for path 'proteinbert/shared_utils'
Cloning into '/content/protein_bert/proteinbert/shared_utils'...
Submodule path 'proteinbert/shared_utils': checked out 'dc1c62a1754c51f6d46b7486e4a3e5e62c0570e1'
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.init

In [3]:
import os
import torch

if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install -q git+https://github.com/snap-stanford/deepsnap.git

import torch_geometric
torch_geometric.__version__

Looking in links: https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 84.5 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.2 MB/s eta 0:00:00


'2.5.3'

In [1]:
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing
import copy
from torch_geometric.utils import degree


from torch import nn, optim, Tensor
from torch_sparse import SparseTensor, matmul

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_gz
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj

from Bio import SeqIO

In [2]:
from proteinbert import load_pretrained_model
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

## Download and Work with String DB

In [4]:
url_links = "https://stringdb-downloads.org/download/protein.physical.links.v12.0/49390.protein.physical.links.v12.0.txt.gz"
url_fasta = "https://stringdb-downloads.org/download/protein.sequences.v12.0/49390.protein.sequences.v12.0.fa.gz"

extract_gz(download_url(url_links, '.'), '.')

extract_gz(download_url(url_fasta, '.'), '.')



Using existing file 49390.protein.physical.links.v12.0.txt.gz
Extracting ./49390.protein.physical.links.v12.0.txt.gz
Using existing file 49390.protein.sequences.v12.0.fa.gz
Extracting ./49390.protein.sequences.v12.0.fa.gz


In [5]:
df = pd.read_csv("49390.protein.physical.links.v12.0.txt",  sep=" ")
df

,protein1,protein2,combined_score
0,49390.A0A068TKE3,49390.A0A068U304,201
1,49390.A0A068TKE6,49390.A0A068UKG9,428
2,49390.A0A068TKE8,49390.A0A068VAR5,360
3,49390.A0A068TKE8,49390.A0A068UNX3,360
4,49390.A0A068TKE8,49390.A0A068UGV5,175
...,...,...,...
997811,49390.Q8LL87,49390.A0A068V4S6,780
997812,49390.Q8LL87,49390.A0A068VGS5,780
997813,49390.Q8LL87,49390.A0A068VGN9,693
997814,49390.Q8LL87,49390.A0A068VDZ3,780


In [6]:
def read_proteins_from_fasta(fasta_file):
    protein_dict = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        protein_dict[record.id] = str(record.seq)
    return protein_dict

In [7]:
protein_sequences_dict = read_proteins_from_fasta("49390.protein.sequences.v12.0.fa")
protein_sequences_dict

{'49390.A0A068TKE3': 'MGHLTAFLHMSNKLADGGHKIFFLLPRKTQAQLKQFNLYPDLIDFIPLCVPQVEGLPPGAETTADIPFHLQPNLRLAMDSTQPQIESILQEIKPQVVFFDFTHWLPKLARRLGIKSIFFITMSSATSGYTFRGEQTTDADLMKAPPGFPSSCIKLLSHEARGLNFAGKVKEIGSGLSFLERLLISAEDCDAIGFKTCREMEGPYCEFIERKFKKPVILAGPVLPEQPTTTLEEKWEKWLSGFKAKSVIFCAFGSECRLQKDQFQELLKGLELTGLPFLAALKPPIGSETIEMALPEGFNERIQGRSVVYGGWVQQQLILAHSSVGCFVTHCGSGSLSEGLVNECQMVLLPQFGDQFINARLMGGDLRVGVEVEKGDEDGLFTKEGVCKAIRMVMDDGSEIGKEVRANHAKWRDFLLREGTESSYIDEFINKMKRLLS',
 '49390.A0A068TKE6': 'MVPYPQAELPLNENDSQEMVLYEVLTEANAISPSNFPSSRNQASVNKMLHRPSHATGKKHYRGVRRRPWGKYAAEIRDSTRNGTRVWLGTFQTAEEAALAYDRAAFRMRGAKALLNFPSEIVTASESSILRQRVHHNHHAKLNLDRQESNSSSSSTEVSSGTGLGLASFESENAAGDLSNAESEEYAATFSVF',
 '49390.A0A068TKE8': 'METRKRGRGVGANREGGRGNQNKYQKNESNFNRGRGGRVGLEENTLAILDTSSSASFHHLLDDRLAFLDAVRSASLVPENDNPPTKKMFVAVFQILKDESSLDLIIASYQLLFELNKRYPWVYLPKMEKSESSTPSKVHCGLIVAEEAWSPFGCNERDEATDTSGGSINPLAFHSLMQDIAKGATEERRNTIEIKSLQNMLLFQYLVNVLEGDLFPRIHAFKESLNWILLRECILNKILGSRKLSYKDLIKDCMSLIGDLSCDKTKITCDQKGKRTSLAELQKDCHGPLELALPEVEK

In [8]:
unique_proteins = set(df['protein1']).union(set(df['protein2']))
unique_proteins

{'49390.A0A068U3V1',
 '49390.A0A068UI60',
 '49390.A0A068U0L1',
 '49390.A0A068TX91',
 '49390.A0A068VMG8',
 '49390.A0A068TWL6',
 '49390.A0A068U418',
 '49390.A0A068UYM2',
 '49390.A0A068UAW3',
 '49390.A0A068UAN2',
 '49390.A0A068VCS2',
 '49390.A0A068UN00',
 '49390.A0A068UGC5',
 '49390.A0A068VH80',
 '49390.A0A068TXM1',
 '49390.A0A068V364',
 '49390.A0A068UCE0',
 '49390.A0A068VLL3',
 '49390.A0A068UJD9',
 '49390.A0A068UUD5',
 '49390.A0A068U0K5',
 '49390.A0A068V557',
 '49390.A0A068TWH5',
 '49390.A0A068UJQ8',
 '49390.A0A068V687',
 '49390.A0A068V479',
 '49390.A0A068TV78',
 '49390.A0A068V4W3',
 '49390.A0A068UH94',
 '49390.A0A068TZ30',
 '49390.A0A068VJZ7',
 '49390.A0A068UQS2',
 '49390.A0A068VR96',
 '49390.A0A068V2M8',
 '49390.A0A068TMR0',
 '49390.A0A068V837',
 '49390.A0A068UH36',
 '49390.A0A068URY9',
 '49390.A0A068TPY0',
 '49390.A0A068TMY5',
 '49390.A0A068UMV1',
 '49390.A0A068U497',
 '49390.A0A068U421',
 '49390.A0A068UXN5',
 '49390.A0A068VEU4',
 '49390.A0A068VMK5',
 '49390.A0A068UPK4',
 '49390.A0A06

In [9]:
filtered_protein_dict = {protein: seq for protein, seq in protein_sequences_dict.items() if protein in unique_proteins}

print(f"Number of matched sequences: {len(filtered_protein_dict)}")
print(f"Number of unique proteins in DataFrame: {len(unique_proteins)}")

Number of matched sequences: 13891
Number of unique proteins in DataFrame: 13891


In [10]:
sequences = list(filtered_protein_dict.values())
protein_names = list(filtered_protein_dict.keys())

longest_sequence_length = max(len(seq) for seq in sequences)
longest_sequence_length

5476

In [ ]:
pretrained_model_generator, input_encoder = load_pretrained_model()

def get_embeddings(seq, seq_len=512, batch_size=1):

    model = get_model_with_hidden_layers_as_outputs(pretrained_model_generator.create_model(seq_len=seq_len))
    encoded_x = input_encoder.encode_X(seq, seq_len)
    local_representations, global_representations = model.predict(encoded_x, batch_size=batch_size)

    return local_representations, global_representations


batch_size = 1  # Adjust based on your GPU memory
seq_len = longest_sequence_length+2
global_embeds = []
local_embeds = []


for i in range(0, len(sequences), batch_size):

    batch_seqs = sequences[i:i + batch_size]
    local_representation, global_representation = get_embeddings(batch_seqs, seq_len=seq_len, batch_size=batch_size)
    global_embeds.extend(global_representation)
    local_embeds.extend(local_representation)

# Convert to numpy array if needed
import numpy as np
global_embeds = np.array(global_embeds)
local_embeds = np.array(local_embeds)

print(f'Global embeddings shape: {global_embeds.shape}')
print(f'Local embeddings shape: {local_embeds.shape}')

1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 12s 12s/step
